In [29]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/publications.csv')

In [2]:
print("\nDATASET STRUCTURE:")
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Time Period: {df['year'].min()} - {df['year'].max()} ({df['year'].max() - df['year'].min() + 1} years)")
print(f"Unique Countries: {df['Name'].nunique()}")
print(f"\nCountries: {', '.join(sorted(df['Name'].unique()))}")



DATASET STRUCTURE:
Shape: 1000 rows × 12 columns
Time Period: 2003 - 2025 (23 years)
Unique Countries: 17

Countries: AUSTRALIA, BRAZIL, CANADA, CHINA, ENGLAND, FRANCE, GERMANY, INDIA, ITALY, JAPAN, NETHERLANDS, SOUTH KOREA, SPAIN, SWEDEN, SWITZERLAND, UNITED KINGDOM, USA


In [3]:

print("KEY PERFORMANCE METRICS:")

metrics = {
    'Total Research Output': df['Web of Science Documents'].sum(),
    'Total Citations': df['Times Cited'].sum(),
    'Avg Citations per Paper': df['Times Cited'].sum() / df['Web of Science Documents'].sum(),
    'Avg CNCI (baseline=1.0)': df['Category Normalized Citation Impact'].mean(),
    'Records with CNCI > 1.0': f"{(df['Category Normalized Citation Impact'] > 1.0).sum()} ({(df['Category Normalized Citation Impact'] > 1.0).mean() * 100:.1f}%)",
    'Avg % Papers in Top 10%': df['% Documents in Top 10%'].mean(),
    'Avg % Papers in Top 1%': df['% Documents in Top 1%'].mean(),
}

for key, value in metrics.items():
    if isinstance(value, float) or isinstance(value, int):
        print(f"{key:.<40} {value:>15.2f}")
    else:
        print(f"{key:.<40} {str(value):>15}")
print("DATA QUALITY:")

missing_total = df.isnull().sum().sum()
if missing_total == 0:
    print("Missing Values: 0 (No missing values detected)")
else:
    print("Missing Values by Column:")
    print(df.isnull().sum())

print(f"Duplicates: {df.duplicated().sum()}")


KEY PERFORMANCE METRICS:
Total Research Output...................        14861699
Total Citations.........................      1296496955
Avg Citations per Paper.................           87.24
Avg CNCI (baseline=1.0).................            1.29
Records with CNCI > 1.0.................     855 (85.5%)
Avg % Papers in Top 10%.................           17.59
Avg % Papers in Top 1%..................            1.77
DATA QUALITY:
Missing Values: 0 (No missing values detected)
Duplicates: 0


In [4]:
print("TOP 5 COUNTRIES BY DIFFERENT METRICS:")

country_agg = df.groupby('Name').agg({
    'Web of Science Documents': 'sum',
    'Times Cited': 'sum',
    'Category Normalized Citation Impact': 'mean',
    '% Documents in Top 1%': 'mean',
    '% Documents in Top 10%': 'mean',
    'Collab-CNCI': 'mean'
}).round(2)

print("\nBy Total Citations:")
top_citations = country_agg.nlargest(5, 'Times Cited')['Times Cited']
for country, value in top_citations.items():
    print(f"   {country}: {value:,.0f}")

print("\nBy Average CNCI:")
top_cnci = country_agg.nlargest(5, 'Category Normalized Citation Impact')['Category Normalized Citation Impact']
for country, value in top_cnci.items():
    print(f"   {country}: {value:.3f}")

print("\nBy Percentage of Documents in Top 1%:")
top_excellence = country_agg.nlargest(5, '% Documents in Top 1%')['% Documents in Top 1%']
for country, value in top_excellence.items():
    print(f"   {country}: {value:.2f}%")


TOP 5 COUNTRIES BY DIFFERENT METRICS:

By Total Citations:
   SPAIN: 98,635,709
   SWITZERLAND: 92,412,437
   BRAZIL: 92,375,787
   CHINA: 85,742,760
   ITALY: 85,321,548

By Average CNCI:
   JAPAN: 1.360
   ITALY: 1.330
   SPAIN: 1.330
   SOUTH KOREA: 1.320
   CANADA: 1.310

By Percentage of Documents in Top 1%:
   UNITED KINGDOM: 1.98%
   BRAZIL: 1.92%
   SWEDEN: 1.92%
   AUSTRALIA: 1.86%
   GERMANY: 1.82%


In [22]:
df[df['Name']=='SPAIN']['Web of Science Documents'].sum()

np.int64(14861699)

In [38]:
df['Name'].unique()

array(['SWITZERLAND', 'CHINA', 'UNITED KINGDOM', 'ITALY', 'SPAIN',
       'AUSTRALIA', 'BRAZIL', 'INDIA', 'CANADA', 'ENGLAND', 'SOUTH KOREA',
       'SWEDEN', 'GERMANY', 'FRANCE', 'JAPAN', 'USA', 'NETHERLANDS'],
      dtype=object)

In [39]:
df.columns

Index(['Name', 'Web of Science Documents', 'Times Cited', 'Collab-CNCI',
       'Rank', '% Docs Cited', 'Category Normalized Citation Impact',
       '% Documents in Top 1%', '% Documents in Top 10%',
       'Documents in Top 1%', 'Documents in Top 10%', 'year'],
      dtype='object')

In [47]:
df['Name'].value_counts()

Name
SPAIN             69
BRAZIL            68
SWITZERLAND       65
CANADA            65
CHINA             65
AUSTRALIA         60
JAPAN             59
FRANCE            59
USA               58
SWEDEN            57
INDIA             56
UNITED KINGDOM    55
ITALY             55
SOUTH KOREA       55
GERMANY           54
ENGLAND           52
NETHERLANDS       48
Name: count, dtype: int64

In [5]:
print("TEMPORAL TRENDS:")

yearly = df.groupby('year').agg({
    'Category Normalized Citation Impact': 'mean',
    'Web of Science Documents': 'sum',
    '% Documents in Top 10%': 'mean'
}).round(2)

cnci_early = yearly.loc[2003:2005, 'Category Normalized Citation Impact'].mean()
cnci_recent = yearly.loc[2023:2025, 'Category Normalized Citation Impact'].mean()
trend = cnci_recent - cnci_early

print("\nCNCI Over Time (First vs Last 3 Years):")
print(f"2003-2005 average CNCI: {cnci_early:.3f}")
print(f"2023-2025 average CNCI: {cnci_recent:.3f}")
print(f"Change: {'+' if trend > 0 else ''}{trend:.3f}")

docs_early = yearly.loc[2003:2005, 'Web of Science Documents'].sum()
docs_recent = yearly.loc[2023:2025, 'Web of Science Documents'].sum()
growth = ((docs_recent / docs_early) - 1) * 100

print("\nOutput Trend:")
print(f"2003-2005 total documents: {docs_early:,.0f}")
print(f"2023-2025 total documents: {docs_recent:,.0f}")

print(f"Growth: {growth:.1f}%")


TEMPORAL TRENDS:

CNCI Over Time (First vs Last 3 Years):
2003-2005 average CNCI: 1.290
2023-2025 average CNCI: 1.290
Change: 0.000

Output Trend:
2003-2005 total documents: 2,178,588
2023-2025 total documents: 2,076,068
Growth: -4.7%


In [6]:
avg_cnci = df['Category Normalized Citation Impact'].mean()
avg_collab_cnci = df['Collab-CNCI'].mean()
collab_advantage = avg_collab_cnci - avg_cnci

In [7]:
print(f"Average CNCI: {avg_cnci:.3f}")
print(f"Average Collab-CNCI: {avg_collab_cnci:.3f}")
print(f"Collaboration Advantage: {'+' if collab_advantage > 0 else ''}{collab_advantage:.3f}")

Average CNCI: 1.292
Average Collab-CNCI: 1.215
Collaboration Advantage: -0.077


In [8]:
positive_collab = (df['Collab-CNCI'] > df['Category Normalized Citation Impact']).sum()
print(f"Records where collaboration improves CNCI: {positive_collab} ({positive_collab / len(df) * 100:.1f}%)")


Records where collaboration improves CNCI: 408 (40.8%)


In [9]:
median_output = df['Web of Science Documents'].median()
high_output = df[df['Web of Science Documents'] > median_output]
low_output = df[df['Web of Science Documents'] <= median_output]

In [10]:
print(f"High-output records average CNCI: {high_output['Category Normalized Citation Impact'].mean():.3f}")
print(f"Low-output records average CNCI: {low_output['Category Normalized Citation Impact'].mean():.3f}")


High-output records average CNCI: 1.294
Low-output records average CNCI: 1.289


In [11]:
high_excellence = df[df['% Documents in Top 1%'] > 2.0]
share_records = len(high_excellence) / len(df) * 100
share_output = high_excellence['Web of Science Documents'].sum() / df['Web of Science Documents'].sum() * 100

In [12]:
print(f"Records with >2% documents in Top 1%: {len(high_excellence)} ({share_records:.1f}%)")
print(f"Share of total output contributed by these records: {share_output:.1f}%")


Records with >2% documents in Top 1%: 399 (39.9%)
Share of total output contributed by these records: 39.0%


In [13]:
print("\nMost Consistent Countries (by CNCI standard deviation):")
consistency = df.groupby('Name')['Category Normalized Citation Impact'].agg(['mean', 'std', 'count'])
consistency = consistency[consistency['count'] >= 5]
consistency = consistency.sort_values('std')
print(consistency.head(5)[['mean', 'std']])


Most Consistent Countries (by CNCI standard deviation):
             mean       std
Name                       
BRAZIL   1.269674  0.205257
ENGLAND  1.239191  0.222746
SPAIN    1.329904  0.223613
ITALY    1.330175  0.224983
JAPAN    1.364284  0.225279


In [48]:
consistent_countries = ['BRAZIL','ENGLAND','SPAIN','ITALY','JAPAN']

for c in consistent_countries:
    sub = df[df['Name']==c]
    print(
        c,
        sub['Category Normalized Citation Impact'].mean(),
        sub['Collab-CNCI'].mean()
    )


BRAZIL 1.2696739117647058 1.2454377794117646
ENGLAND 1.2391907692307693 1.2056513846153847
SPAIN 1.3299043913043476 1.1878204057971014
ITALY 1.3301749454545455 1.1878988363636362
JAPAN 1.3642843898305081 1.1842726101694916


## Deciding STORY ANGLES

In [14]:
below_average_records = (df['Category Normalized Citation Impact'] < 1.0).sum()
below_average_share = (df['Category Normalized Citation Impact'] < 1.0).mean() * 100
max_cnci = df['Category Normalized Citation Impact'].max()
avg_top1 = df['% Documents in Top 1%'].mean()

In [15]:
print("\n1. The Quality Distribution")
print(f"   Records with CNCI < 1.0: {below_average_records} ({below_average_share:.0f}%)")
print(f"   Highest observed CNCI: {max_cnci:.2f}")


1. The Quality Distribution
   Records with CNCI < 1.0: 145 (14%)
   Highest observed CNCI: 1.70


In [16]:
print("\n2. Collaboration Effect")
print(f"   Average CNCI gain from collaboration: {collab_advantage:.3f}")

print("\n3. Evolution Over Two Decades")
print(f"   Research output growth from 2003-2005 to 2023-2025: {growth:.0f}%")
print(f"   Average CNCI over the same periods has {'improved' if trend > 0 else 'declined'} by {abs(trend):.3f}")

print("\n4. Excellence Gap")
print(f"   Top country share in Top 1% documents: {top_excellence.iloc[0]:.2f}%")
print(f"   Overall average share in Top 1% documents: {avg_top1:.2f}%")



2. Collaboration Effect
   Average CNCI gain from collaboration: -0.077

3. Evolution Over Two Decades
   Research output growth from 2003-2005 to 2023-2025: -5%
   Average CNCI over the same periods has declined by 0.000

4. Excellence Gap
   Top country share in Top 1% documents: 1.98%
   Overall average share in Top 1% documents: 1.77%
